In [ ]:
#CELL DEDICATED TO IMPORT PACKAGES USEFUL FOR OUR PURPOSES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import initializers
from keras.callbacks import EarlyStopping
from keras.callbacks import History
from keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix
from keras.models import Model
from keras.preprocessing import image
from keras.layers import Conv2D,Flatten,Dropout, MaxPooling2D,AveragePooling2D
from keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import time 

import import_ipynb
import utils


In [ ]:
# SOME IMPORTANT GLOBAL VARIABLE
labels = ["Bedroom","Coast","Forest","HighWay","Industrial","InsideCity","Kitchen","LivingRoom","Mountain","Office","OpenCountry","Store","Street","Suburb","TallBuilding"]
train_dir = '../images/train/'
test_dir = '../images/test/'
from keras.preprocessing import image

# CREATION OF TRAINING DATA

In a similar way to the previous parts, I preprocess my data; this time I have to take into account that images have to be suitable for VGG, so for example images will be of (224,224).



In [ ]:
'''Fit data to make them suitable'''
le = LabelEncoder()

In [ ]:
list_of_images = utils.list_of_path(labels,train_dir)
train_data,train_labels = utils.read_and_process_images(list_of_images,dimension=224)

In [ ]:
train_images =  []
for im in list_of_images:
    temp_img=image.load_img(im,target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    train_images.append(temp_img)

    

train_images=np.array(train_images)
train_img=(train_images)


train_labels = le.fit_transform(train_labels)
train_labels_dummy=to_categorical(train_labels,15)

# CREATION OF TEST DATA

Same thing with test data.

In [ ]:
list_of_images = utils.list_of_path(labels,test_dir)
test_data,test_labels = utils.read_and_process_images(list_of_images,dimension=224)




from keras.applications.vgg16 import preprocess_input
test_images =  []

for im in list_of_images:
    temp_img=image.load_img(im,target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    test_images.append(temp_img)

    
test_images=np.array(test_images)
test_img=preprocess_input(test_images)




test_labels=le.fit_transform(test_labels)
test_labelsdummy=to_categorical(test_labels,15)

In [ ]:
def vgg16_model(img_rows, img_cols, channel=1, num_classes=15):
    """
    Function which initialize a model which is equal to the vgg_16, with the last classification
    layer adapted for our problem (15 classification layer)
    
    Input img_rows = x dimension of images
    Input img_cols = y dimension of images
    Input channel(1)= number of channels of the images
    Input num_classes(15) = number of classes to classify
    
    Output model_res =  VGG16 model
    """
    adam = optimizers.Adam()
    model = VGG16(weights='imagenet', include_top=True)
    model_res = Sequential()
    for layer in model.layers[:-1]: # just exclude last layer from copying
        model_res.add(layer)
    
    model_res.add(Dense(15,activation='softmax'))
    for layer in model_res.layers[:-1]:

        layer.trainable = False
    model_res.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    return model_res

In [ ]:
model = vgg16_model(224, 224, 3)

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
print("My Custom CNN Network:")
plot_model(model, to_file='Images_point_three/model_tl.png', show_shapes=True, show_layer_names=True)

In [ ]:
'''Training the model'''
history = History()
earlyStopping = EarlyStopping(min_delta=0.00,patience = 3)
start_time_1 = time.time()
model.fit(train_img,train_labels_dummy,batch_size=64,epochs=10,validation_split=0.15,shuffle=True,callbacks=[earlyStopping,history])
end_time_1 = time.time()

In [ ]:
timing_svv = utils.convert(end_time_1-start_time_1)
timing_svv

In [ ]:
'''Evaluating the model'''
eval_time_start = time.time()
ev = model.evaluate(test_img,test_labelsdummy)
eval_time_end = time.time()

In [ ]:
ev

In [ ]:
tim = utils.convert(eval_time_end - eval_time_start)

In [ ]:
tim

In [ ]:
'''I predict classes of my test_data in order to build my confusion matrix''' 
y_pred = model.predict_classes(test_img)

In [ ]:
cm = confusion_matrix(test_labels,y_pred)

In [ ]:
plt.figure(figsize=(14,7)) 
utils.plot_confusion_matrix(cm,labels,"point_three","images_point_three/",normalize=True)

In [ ]:
plt.title("validation loss")
plt.xlabel("epochs")
plt.ylabel("val-loss")
plt.plot(history.history['val_loss'])
plt.grid()
plt.savefig("images_point_three/validation_loss_3.jpg")


In [ ]:
plt.title("validation accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history.history['val_accuracy'])
plt.grid()
plt.savefig("images_point_three/validation_accuracy_3.jpg")


In [ ]:
plt.title(" accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history.history['accuracy'])
plt.grid()
plt.savefig("images_point_three/accuracy_3.jpg")

In [ ]:
plt.title("Loss")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history.history['loss'])
plt.grid()
plt.savefig("images_point_three/loss_3.jpg")